# core

A set of core functions and a wrapper for the azure cli and some other azure apis, which simplifies authentication and retrying

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
import json, subprocess, os, sys, pandas
from platformdirs import PlatformDirs
from diskcache import Cache, memoize_stampede
from tenacity import wait_random_exponential, stop_after_attempt, Retrying

## Standardised default disk caching and retry helpers
The below `cache` sets up a persistent per user disk cache (to ensure security) that can be used throughout api setup and configuration. `retryer` will try to run a function again up to 3 times with a random exponential backoff to handle upstream api exceptions.

In [ ]:
#| export
dirs = PlatformDirs("nbdev-squ")
cache = Cache(dirs.user_cache_dir)
retryer = Retrying(wait=wait_random_exponential(), stop=stop_after_attempt(3))

In [ ]:
#| export
def _cli(cmd: list[str], capture_output=True):
    cmd = [sys.executable, "-m", "azure.cli"] + cmd + ["-o", "json"]
    if capture_output: # Try lots, parse output as json
        result = retryer(subprocess.run, cmd, capture_output=capture_output, check=True)
        return json.loads(result.stdout.decode("utf8"))
    else: # Run interactively, ignore success/fail
        subprocess.run(cmd)

## Login and secrets management
The squ library depends on authentication configured and ready to go. There are 2 paths to login used based on environment variables available.

In [ ]:
#| export
def login(refresh: bool=False # Force relogin
         ):
    if os.environ.get("IDENTITY_HEADER") and not cache.get("msi_failed"):
        if refresh:
            _cli(["logout"])
        try:
            _cli(["login", "--identity"])
        except subprocess.CalledProcessError:
            cache["msi_failed"] = True
        else:
            cache.delete("msi_failed")
            cache["logged_in"] = True
            cache["login_time"] = pandas.Timestamp("now")
    if not os.environ.get("IDENTITY_HEADER") or cache.get("msi_failed"):
        while not cache.get("logged_in"):
            try:
                _cli(["account", "show"])
            except subprocess.CalledProcessError:
                tenant = cache.get("tenant", [])
                if tenant:
                    tenant = ["--tenant", tenant]
                _cli(["login", *tenant, "--use-device-code"], capture_output=False)
            else:
                cache["logged_in"] = True
                cache["login_time"] = pandas.Timestamp("now")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()